Base model case

1) generating recommendations and counting hitrate score

In [6]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np

CONN = "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"\
    "postgres.lab.karpov.courses:6432/startml"

model = CatBoostClassifier()
model_path = '../airflow/dags/recommendation_models/catboost_base_model'

model.load_model(model_path)

In [7]:
user_info = pd.read_sql(
    """SELECT * FROM public.user_data""",
    con=CONN
)
print(user_info.shape)
user_info.sample(5,random_state=0)

(163205, 8)


,user_id,gender,age,country,city,exp_group,os,source
10525,10728,1,45,Russia,Chelyabinsk,4,iOS,ads
104822,110152,1,23,Ukraine,Melitopol,1,Android,organic
161513,166858,0,28,Russia,Groznyy,2,Android,organic
118361,123691,1,18,Russia,Novosibirsk,2,Android,organic
33304,33525,0,19,Russia,Kireyevsk,4,Android,ads


In [9]:
views = pd.read_csv('views.csv')
views['date_s'] = pd.to_datetime(views.timestamp, unit='s')
likes = pd.read_csv('likes.csv')
likes['date_s'] = pd.to_datetime(likes.timestamp, unit='s')

In [10]:
tmp = views.groupby('user_id').exp_group.nunique().reset_index()

duplicate_users = tmp[tmp.exp_group > 1].user_id.values

views = views[~np.in1d(views.user_id, duplicate_users)]
likes = likes[~np.in1d(likes.user_id, duplicate_users)]

In [11]:
posts_features = pd.read_sql(
    """SELECT * FROM public.grokhi_base_model_posts_info_features""",
    con=CONN
)

posts_features = posts_features.drop(
    ['index', 'text', 'topic'], axis=1
)

In [12]:
def get_recommendations(
    user_id:pd.Series, 
    time:pd.Series, 
    user_features:pd.DataFrame, 
    likes:pd.DataFrame, 
    limit:int = 5
) -> list:
    # user_id = views.loc[0,'user_id']

    user_features = user_features[user_features.user_id==user_id].drop('user_id', axis=1)

    add_user_features = dict(zip(user_features.columns, user_features.values[0]))

    user_posts_features = posts_features.assign(**add_user_features)
    user_posts_features = user_posts_features.set_index('post_id')

    user_posts_features['hour'] = time.hour
    user_posts_features['month'] = time.month

    predicts = model.predict_proba(user_posts_features)[:, 1]
    user_posts_features['predicts'] = predicts

    # deleting liked posts
    liked_posts = likes[likes.user_id == user_id].post_id.values

    filtered_ = user_posts_features[~user_posts_features.index.isin(liked_posts)]
    recommended_posts = filtered_.sort_values('predicts')[-limit:].index

    return recommended_posts.to_list()

In [13]:
views_ = views.drop(columns='recommendations').copy()[:1000]

views_['recommendations'] = views_.apply(
    lambda row: get_recommendations(
        row['user_id'], 
        row['date_s'],
        user_features=user_info,
        likes=likes
    ), axis=1
)
views_

,user_id,exp_group,timestamp,date_s,recommendations
0,128381,control,1654030803,2022-05-31 21:00:03,"[2776, 2782, 7122, 5765, 3834]"
1,146885,test,1654030811,2022-05-31 21:00:11,"[1491, 1742, 1830, 7296, 2904]"
2,50948,test,1654030825,2022-05-31 21:00:25,"[1830, 1742, 1253, 7296, 2904]"
3,37703,test,1654030826,2022-05-31 21:00:26,"[1884, 7296, 1742, 1253, 2904]"
4,14661,test,1654030829,2022-05-31 21:00:29,"[1491, 1742, 2904, 1884, 1218]"
...,...,...,...,...,...
995,51158,test,1654037179,2022-05-31 22:46:19,"[3834, 5765, 3677, 4052, 2782]"
996,69954,control,1654037179,2022-05-31 22:46:19,"[249, 1766, 576, 466, 213]"
997,142789,control,1654037191,2022-05-31 22:46:31,"[7296, 576, 1766, 466, 213]"
998,84424,test,1654037196,2022-05-31 22:46:36,"[624, 6966, 5370, 5321, 5950]"


In [14]:
tmp = pd.merge(views_, likes, on='user_id', how='outer')
tmp.post_id = tmp.post_id.fillna(-1).astype(int)

tmp = tmp.dropna()

In [15]:
tmp.recommendations = tmp.recommendations.astype(str)
tmp['recommendations'] = tmp.recommendations.apply(
    lambda x: list(map(int, filter(bool, x[1:-1].split(', '))))
)

tmp.sample(5)[['post_id','recommendations']]

,post_id,recommendations
3492,6890,"[2388, 3853, 5765, 2776, 3834]"
4830,5456,"[624, 5474, 5370, 5321, 5950]"
3749,4787,"[1176, 1161, 1830, 7296, 2904]"
5231,2557,"[7296, 576, 1766, 466, 213]"
2656,7014,"[1730, 576, 1766, 466, 213]"


In [ ]:
tmp.post_id = tmp.apply(
    lambda row:
    -1
    if
        (row.post_id == -1) | 
        ((row.timestamp_x > row.timestamp_y) |
        (row.timestamp_x + 60 * 60 < row.timestamp_y)) |
        (row.post_id not in row.recommendations)
    else
    row.post_id, axis=1
)

tmp.sample(5)[['post_id','recommendations']]

In [ ]:
def agg_hitrate(values):
    values = set(values)
    if -1 in values and len(values) >= 2:
        return 1
    elif -1 not in values:
        return 1
    return 0

tmp_agg = (
    tmp.groupby(['user_id', 'exp_group', 'timestamp_x'])
    .post_id.agg(agg_hitrate)
)
tmp_agg = tmp_agg.reset_index().rename(
    columns={'post_id': 'hitrate'}
)

print('Getting Hitrate...')
hitrate_score = tmp_agg.hitrate.mean()
hitrate_score

Getting Hitrate...


0.0

Oops, recommendations not working properly. Will change that in future. For now, let's use provided recommendations column from KarpovCourses

In [18]:
tmp = pd.merge(views, likes, on='user_id', how='outer')
tmp.post_id = tmp.post_id.fillna(-1).astype(int)

tmp.recommendations = tmp.recommendations.astype(str)
tmp['recommendations'] = tmp.recommendations.apply(
    lambda x: list(map(int, filter(bool, x[1:-1].split(' '))))
)

tmp.sample(5)[['post_id','recommendations']]
tmp.post_id = tmp.apply(
    lambda row:
    -1
    if
        (row.post_id == -1) | 
        ((row.timestamp_x > row.timestamp_y) |
        (row.timestamp_x + 60 * 60 < row.timestamp_y)) |
        (row.post_id not in row.recommendations)
    else
    row.post_id, axis=1
)

tmp.sample(5)[['post_id','recommendations']]
def agg_hitrate(values):
    values = set(values)
    if -1 in values and len(values) >= 2:
        return 1
    elif -1 not in values:
        return 1
    return 0

tmp_agg = (
    tmp.groupby(['user_id', 'exp_group', 'timestamp_x'])
    .post_id.agg(agg_hitrate)
)
tmp_agg = tmp_agg.reset_index().rename(
    columns={'post_id': 'hitrate'}
)

print('Getting Hitrate...')
hitrate_score = tmp_agg.hitrate.mean()
hitrate_score

Getting Hitrate...


0.7132582734855227

2) get abtest table and make A/B Test

In [19]:
# Bucketing users
import hashlib

tmp_agg['bucket'] = tmp_agg['user_id'].apply(
    lambda x: int(hashlib.md5((str(x) + 'bbb').encode()).hexdigest(), 16) % 100
)

tmp_agg['view'] = 1

new_df = (
    tmp_agg
    .drop(columns=['timestamp_x', 'user_id' ])
    .groupby(['exp_group', 'bucket'])
    .sum(['hitrate', 'view'])
    .reset_index()
)
new_df['hitrate_new'] = new_df.hitrate / new_df.view

new_df.head()

,exp_group,bucket,hitrate,view,hitrate_new
0,control,0,621,890,0.697753
1,control,1,666,941,0.707758
2,control,2,705,1006,0.700795
3,control,3,645,921,0.700326
4,control,4,710,969,0.732714


In [20]:
tmp_agg.groupby('exp_group').hitrate.mean()

exp_group
control    0.706651
test       0.719829
Name: hitrate, dtype: float64

In [21]:
from scipy.stats import mannwhitneyu, ttest_ind

mannwhitneyu(
    new_df[new_df.exp_group == 'control'].hitrate_new,
    new_df[new_df.exp_group == 'test'].hitrate_new,
)

MannwhitneyuResult(statistic=2474.0, pvalue=6.795052834951048e-10)

In [22]:
ttest_ind(
    new_df[new_df.exp_group == 'control'].hitrate_new,
    new_df[new_df.exp_group == 'test'].hitrate_new,
)

Ttest_indResult(statistic=-6.283293948472979, pvalue=2.0664050599155474e-09)

Null hypothesis of mean hitrates equality is rejected! Change is statistically significant!